<a href="https://colab.research.google.com/github/ElviraKonovalov/soen471-bigData/blob/main/DataCleaning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [169]:
import pandas as pd
import matplotlib.pyplot as plt
import datetime

In [170]:
file = '../data/Motor Vehicle Collisions with KSI Data.csv'
df=  pd.read_csv(file, sep=',');

columns_to_drop = ['_id', 'ACCNUM', 'TIME', 'OFFSET', 'WARDNUM', 'DIVISION', 'ACCLOC', 'FATAL_NO', 'PEDTYPE', 'PEDACT', 'PEDCOND', 'CYCLISTYPE', 'CYCACT', 'CYCCOND', 'POLICE_DIVISION', 'ObjectId', 'geometry']
df = df.drop(columns=columns_to_drop)

/tmp/ipykernel_23549/3376709844.py:2: DtypeWarning: Columns (16) have mixed types. Specify dtype option on import or set low_memory=False.
  df=  pd.read_csv(file, sep=',');


Features 1 - 8

In [172]:
#replace fatal with 1, non fatal with 0

df['ACCLASS'] = df['ACCLASS'].replace(['Fatal'], 1)
df['ACCLASS'] = df['ACCLASS'].replace(['Non-Fatal Injury'], 0)
#drop rows (2) where there was only property Damage
df.drop(df.index[df['ACCLASS'] == 'Property Damage Only'], inplace=True)


df.AG_DRIV.fillna(0, inplace=True)
df['AG_DRIV'] = df['AG_DRIV'].replace(['Yes'], 1)


df.ALCOHOL.fillna(0, inplace=True) #replaces NA to 0
df['ALCOHOL'] = df['ALCOHOL'].replace(['Yes'], 1) #replaces YES to 1

df['AUTOMOBILE'] = df['AUTOMOBILE'].fillna(0)
df['AUTOMOBILE'].replace('Yes',1, inplace=True)

df['CYCLIST'] = df['CYCLIST'].fillna(0)
df['CYCLIST'].replace('Yes',1, inplace=True)


#first we will have to convert the data type to DateTime
df['DATE'] = pd.to_datetime(df['DATE'])

#now we will transform the data in the DATE column to simply present the month in integer form
df['DATE'] = df['DATE'].dt.month


df.DISABILITY.fillna(0, inplace=True) #replaces NA to 0
df['DISABILITY'] = df['DISABILITY'].replace(['Yes'], 1) #replaces YES to 1


df = df.drop(columns='DISTRICT') #drop district


df = df.drop(columns='DRIVACT') #drop driver action

Features 9 - 18

In [173]:
df = df.drop(columns = ['DRIVCOND'])

df['EMERG_VEH'] = df['EMERG_VEH'].fillna(0)
df['EMERG_VEH'].replace('Yes',1, inplace=True)



df = df.drop(columns = ['IMPACTYPE'])


df.INITDIR.fillna("Unknown", inplace = True)


df.INVAGE = df.INVAGE.replace(['unknown'], 'Unknown') #All of the unknowns in this dataset are upper case except the ones in this collumn so it was bugging me.

#df = df.drop(columns = ['INVAGE'])


df.INVTYPE.fillna('Unknown', inplace=True)

#Removing rows where they weren't hit, as they obviously wouldn't ever have any injuries.

rows = df.INVTYPE.unique()

#['Cyclist', 'Cyclist Passenger', 'Motorcycle Passenger', 'Passenger','In-Line Skater', 'Trailer Owner', 'Other Property Owner', 'Vehicle Owner', 'Other', 'Wheelchair', 'Pedestrian', 'Unknown', 'Witness', 'Pedestrian - Not Hit', 'Driver - Not Hit']

#Remove rows I don't like
for row in rows:
  if 'Driver' not in row or 'Not Hit' in row:
    df = df[(df.INVTYPE != row)] 

#Replacing not necesarry.
#df.INVTYPE = df.INVTYPE.replace(['Motorcycle Driver', 'Moped Driver', 'Truck Driver'], 'Driver')

#Dropping the column now that I've filtered rows.
df = df.drop(columns = ['INVTYPE'])

Features 18 - 26

In [174]:
columns_to_drop = ['NEIGHBOURHOOD']
df = df.drop(columns=columns_to_drop)


#LIGHT
df.LIGHT.unique()

# Dropping all rows that have "Other" as value

df = df.drop(df[df.LIGHT == 'Other'].index)
df = df.drop(df[df.LIGHT.isnull()].index)



#LOCCOORD
df.LOCCOORD.unique()
df.LOCCOORD.isnull().sum() #105


#PEDESTRIAN
#replace every "NaN" to 0 and "Yes" to 1


df['PEDESTRIAN'] = df['PEDESTRIAN'].fillna(0)
df['PEDESTRIAN'].replace(('Yes'),(1), inplace=True)



#PASSENGER
#replace every "NaN" to False and "Yes" to True

df['PASSENGER'] = df['PASSENGER'].fillna(0)
df['PASSENGER'].replace(('Yes'),(1), inplace=True)



#MOTORCYCLE
#replace every "NaN" to False and "Yes" to True

df['MOTORCYCLE'] = df['MOTORCYCLE'].fillna(0)
df['MOTORCYCLE'].replace(('Yes'),(1), inplace=True)




df.RDSFCOND.isnull().sum() # 23
df = df.drop(df[df.RDSFCOND.isnull()].index)


#REDLIGHT
#replace every "NaN" to False and "Yes" to True

df['REDLIGHT'] = df['REDLIGHT'].fillna(0)
df['REDLIGHT'].replace(('Yes'), (1), inplace=True)



df['MANOEUVER'] = df['MANOEUVER'].fillna('Unknown')



Features 27 - 37

In [175]:
'''
"Every street owned by the City of Toronto has been given one of five classifications (with the exception of public laneways)."
classes: Local road, Collector road, Minor arterial road, Major arterical road and Expressway
https://www.toronto.ca/services-payments/streets-parking-transportation/traffic-management/road-classification-system/about-the-road-classification-system/
'''
# replace nan values with most frequent value in the column
df['ROAD_CLASS'] = df['ROAD_CLASS'].fillna(df['ROAD_CLASS'].value_counts().index[0])
# drop rows where ROAD_CLASS is pending
df.drop(df.index[df['ROAD_CLASS'] == 'Pending'], inplace=True)
# replacing Expressway Ramp (4 rows) with Expressway
df['ROAD_CLASS'] = df['ROAD_CLASS'].replace(['Expressway Ramp'],'Expressway')
# Major Arterial Ramp (1 row) with Major Arterial
df['ROAD_CLASS'] = df['ROAD_CLASS'].replace(['Major Arterial Ramp'],'Major Arterial')



# replacing nan with 0 ('No') and 'Yes' with 1
df['SPEEDING'] = df['SPEEDING'].fillna(0)
df['SPEEDING'] = df['SPEEDING'].replace(['Yes'], 1)


# drop street names
df = df.drop(columns=['STREET1', 'STREET2'])


# group all into no control, passive control or active control
df['TRAFFCTL'] = df['TRAFFCTL'].replace(['Traffic Signal', 'Stop Sign', 'Pedestrian Crossover', 'Yield Sign', 'Streetcar (Stop for)', 'Traffic Gate'], 'Passive Control')
df['TRAFFCTL'] = df['TRAFFCTL'].replace(['Police Control', 'School Guard', 'Traffic Controller'], 'Active Control')
df['TRAFFCTL'] = df['TRAFFCTL'].fillna('Passive Control')


# replace nan with 0 ('No') and 'Yes' with 1
df['TRSN_CITY_VEH'] = df['TRSN_CITY_VEH'].fillna(0)
df.TRSN_CITY_VEH.replace(('Yes'), (1), inplace=True)


# replace nan with 0 ('No') and 'Yes' with 1
df['TRUCK'] = df['TRUCK'].fillna(0)
df.TRUCK.replace(('Yes'), (1), inplace=True)


df = df.drop(columns='VEHTYPE') # drop





df = df.drop(columns='YEAR') # drop year

In [176]:
df = df.drop(df[df.VISIBILITY.isnull()].index)

In [177]:
df
# save csv no hot-one encoding
df.to_csv("../data/clean_data.csv")

In [178]:

# label encoding
df['ACCLASS'] = df['ACCLASS'].astype('category').cat.codes
df['INITDIR'] = df['INITDIR'].astype('category').cat.codes
df['LIGHT'] = df['LIGHT'].astype('category').cat.codes
df['VISIBILITY'] = df['VISIBILITY'].astype('category').cat.codes
df['RDSFCOND'] = df['RDSFCOND'].astype('category').cat.codes
df['ROAD_CLASS'] = df['ROAD_CLASS'].astype('category').cat.codes # label encoding
df['TRAFFCTL'] = df['TRAFFCTL'].astype('category').cat.codes



In [179]:
df
# save csv with hot-one encoding
df.to_csv("../data/clean_data_hot-one.csv")